In [12]:
import cv2
import numpy as np

# 读取图像并进行灰度化处理
image = cv2.imread('image.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 对图像进行二值化处理
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# 对二值图像进行形态学操作，以去除噪声
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

# 进行背景区域的确定
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# 寻找前景区域
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)

# 找到不确定的区域
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# 标记分水岭算法的标签
_, markers = cv2.connectedComponents(sure_fg)

# 增加标签以区分不确定的区域
markers = markers + 1
markers[unknown == 255] = 0

# 应用分水岭算法
markers = cv2.watershed(image, markers)
image[markers == -1] = [0, 0, 255]  # 标记分割边界为红色

# 显示结果
cv2.imshow('Original Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()